####import packages

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDClassifier, LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_validate
import xgboost as xgb
import lightgbm as lgb

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning, module="my_module")
import warnings
warnings.simplefilter('ignore')


In [ ]:
df = pd.read_csv('merged_df(window size 0.04 approximately) (3).csv')
replacement_dict = {'Saccade': 0, 'Fixation': 1, 'Unclassified': 2, 'EyesNotFound': 3}

# Replace the values in the column
df['Eye movement type'] = df['Eye movement type'].replace(replacement_dict)

feature_names = ['Gaze event duration', 'Fixation point X',
       'Fixation point Y', 'Eye movement type', 'Accelerometer X',
       'Accelerometer Y', 'Accelerometer Z', 'Gyro X', 'Gyro Y', 'Gyro Z',
       'Gaze point X', 'Gaze point Y', 'Gaze point 3D X', 'Gaze point 3D Y',
       'Gaze point 3D Z', 'Gaze direction left X', 'Gaze direction left Y',
       'Gaze direction left Z', 'Gaze direction right X',
       'Gaze direction right Y', 'Gaze direction right Z',
       'Pupil position left X', 'Pupil position left Y',
       'Pupil position left Z', 'Pupil position right X',
       'Pupil position right Y', 'Pupil position right Z',
       'Pupil diameter left', 'Pupil diameter right']
X = df[feature_names]
imputer = SimpleImputer()
X_filled = imputer.fit_transform(X)
X_filled = pd.DataFrame(X_filled, columns=feature_names)
X_filled['Time Window'] = df['Time Window']
X_filled['is_distraction'] = df['is_distraction']
X_filled['videoNum'] = df['videoNum']
X_filled['Participant ID'] = df['Participant ID']
X_filled['order'] = df['order']
X_filled['interest_level'] = df['interest_level']


X_filled = X_filled[X_filled['Time Window'].notna()]

In [ ]:
def parse_data(df, window_size, include = True):
### THE function for data window parsing
### return 1) parsed window contains EITHER distraction or attention
### return 2) parsed window contains neigher distraction nor attention, and not overlap with 1)


  all_windows = [] # windows for verbolization
  all_windows_neg = []
  tracked = False
  last_time = None
  last_index = -1
  videoN = -1
  for index, row in df.iloc[::-1].iterrows():
    if tracked ==True:
      # print('Here')
      if row['is_distraction']!='None' or row['videoNum']!=videoN:
        res = row['Time Window'].strip('][').split(', ')
        # print(last_time, float(res[0]))
        if ((last_time - float(res[0])) <= window_size):
          if include and (last_index-index>3):
            all_windows.append(df.iloc[index+1:last_index+2].copy())
        last_time = float(res[1])
        last_index = index
        if row['videoNum']!=videoN and row['is_distraction']=='None':
          tracked=False
          # print(1, last_time-float(res[0]))
          # print('find a window')
        videoN = row['videoNum']
      elif row['is_distraction']=='None':
        res = row['Time Window'].strip('][').split(', ')
        if row['videoNum']!=videoN:
          tracked = False
          last_index = index
          videoN = row['videoNum']
        if (last_time - float(res[0])) > window_size:
          all_windows.append(df.iloc[index+1:last_index+1].copy())
          tracked = False
          last_index = index
          videoN = row['videoNum']
          # print(2, last_time-float(res[0]))

          # print('find a window')
    else:
      if row['is_distraction']!='None':
        # print('window')
        res = row['Time Window'].strip('][').split(', ')
        tracked = True
        last_time = float(res[1])
        all_windows_neg.append(df.iloc[index+1:last_index])
        last_index = index
        videoN = row['videoNum']

  ## negation of window data
  finite_all_windows_neg = []
  for wind in all_windows_neg:
    if wind.shape[0]>0:
      start_time = float(wind.iloc[0]['Time Window'].strip('][').split(', ')[0])
      for index, row in wind.iterrows():
        t = float(row['Time Window'].strip('][').split(', ')[1])
        if (t-start_time)>=window_size:
          finite_all_windows_neg.append(wind.loc[0:index-1].copy())
          break
  print(len(all_windows), len(finite_all_windows_neg))
  return all_windows, finite_all_windows_neg

def parse_neg_data(df, window_size):
  all_windows_neg=[]
  start_index = df.shape[0]-1
  start_time = float(df.iloc[start_index]['Time Window'].strip('][').split(', ')[1])
  videoN = df.iloc[start_index]['videoNum']
  for index, row in df.iloc[::-1].iterrows():
    if row['is_distraction']!='None' or (videoN!=row['videoNum']):
      start_index = index-1
      start_time = float(df.iloc[start_index]['Time Window'].strip('][').split(', ')[1])
    if ((start_time-float(df.iloc[index]['Time Window'].strip('][').split(', ')[0]))>window_size) and (start_index-index>3):
      all_windows_neg.append(df.iloc[index+1:start_index+1].copy())
      start_index = index
      start_time = float(df.iloc[start_index]['Time Window'].strip('][').split(', ')[1])
  return all_windows_neg

def check_windows(parsed_df):
  checked = False
  for i, df in enumerate(parsed_df):
    a, b = df[df['is_distraction']=='True'].shape
    c, d = df[df['is_distraction']=='False'].shape
    if a>0 and c>0:
      print('Found Irregular Df:',i)
      checked=True
    if a==0 and c==0:
      print('Found no label Df:',i)
      checked=True
  if not checked:
    print('No irregular rows')

def check_neg_windows(parsed_df, window_size, tol):
  checked = False
  for i, df in enumerate(parsed_df):
    a, b = df[df['is_distraction']!='None'].shape
    s = float(df.iloc[0]['Time Window'].strip('][').split(', ')[0])
    e = float(df.iloc[-1]['Time Window'].strip('][').split(', ')[1])
    if a>0:
      print('Contains Distraction Data',i)
      checked = True
    if np.abs(window_size - (e-s)) > tol:
      print('Inrregular Large Window',i)
      checked = True
  if not checked:
      print('No irregular rows')

####Function

In [ ]:
#std
def get_std_df(df_list, column_names):
    std_list = []

    for df in df_list:
        std_row = []
        for col in column_names:
            std_row.append(np.std(df[col]))
        std_list.append(std_row)

    std_df = pd.DataFrame(std_list, columns=column_names)

    return std_df


In [ ]:
#sum
def calculate_avg_a(df_list, column_names):
    l1 = []
    for df in df_list:
        df = df.reset_index(drop=True)
        a_list = []
        for i in range(len(df[column_names[0]])):
            if i > 0:
                a = np.sqrt(np.square(df[column_names[0]][i] - df[column_names[0]][i-1]) \
                + np.square(df[column_names[1]][i] - df[column_names[1]][i-1]) \
                + np.square(df[column_names[2]][i] - df[column_names[2]][i-1]))
                a_list.append(a)
        sum_a = np.sum(a_list)
        l1.append(sum_a)
    return pd.DataFrame({'avg_a': l1})

In [ ]:
def Saccade_fixation_ratio(df_list):
    fixation_count = []
    saccade_count = []

    for df in df_list:

        fixation = df[df['Eye movement type'] == 0]['Eye movement type'].count()
        saccade = df[df['Eye movement type'] == 1]['Eye movement type'].count()

        fixation_count.append(fixation/len(df))
        saccade_count.append(saccade/len(df))

    result = pd.DataFrame({'Fixation count': fixation_count, 'Saccade count': saccade_count})

    return result

In [ ]:
def absolute_changes(df_list, col_name):
  max = 0
  min = 0
  a = 0
  change = []
  for df in df_list:
    for i in df[col_name]:
      if i < min:
        min = i
      elif i > max:
        max = i
    a = abs(max - min)
    min = 0
    max = 0
    change.append(a)

  return pd.DataFrame({'abs_change': change})

In [ ]:
def avg_col(df_list, col_name):
  y = []
  a1 = []
  for df in df_list:
    for i in df[col_name]:
      y.append(i)
    average = abs(np.mean(y))
    y = []
    a1.append(average)
  return pd.DataFrame({'avg1': a1})

In [ ]:
def con_df(df1, df2, df3):
  #concatenated = df1.join(df2)
  concatenated = df1.join([df2, df3])
  return concatenated

In [ ]:
def longest_fixation(df_list1):
    b = 0
    c = 0
    d = list(range(2390))
    e = []
    f = 0
    a = []
    b = 0
    #fixation_range_container = []

    for df in df_list1:
        b += 1
        #print(b)
        last_index_label = df.index.max()
        last_row = df.loc[last_index_label]
        start_time = 0
        fixation_range_container = []
        for index, row in df.iterrows():
            i = row['Eye movement type']
            j = row['Time Window']

            if i == 1 and not row.equals(last_row):
                res = j.strip('][').split(', ')
                last_time = float(res[1])
                first_time = float(res[0])
                if start_time == 0:
                    start_time = first_time

            if i != 1 and not row.equals(last_row):
                res = j.strip('][').split(', ')
                first_time_2 = float(res[0])
                if start_time != 0:
                    fixation_range_container.append(first_time_2 - start_time)
                    start_time = 0

            if row.equals(last_row):
                if i == 1:
                    res = j.strip('][').split(', ')
                    last_time = float(res[1])
                    first_time = float(res[0])
                    if start_time == 0:
                      fixation_range_container.append(last_time - first_time)

                      start_time = 0
                      #a.append(max(fixation_range_container))
                      #e.append(len(a))
                    else:
                      fixation_range_container.append(last_time - start_time)
                      #if b-1 == 53:
                        #print(fixation_range_container)
                      #print(b-1, fixation_range_container)
                    a.append(max(fixation_range_container))
                      #e.append(len(a))
                    #if b-1 == 53:
                      #print(a[-1])
                    #print(index)
                else:
                    if start_time != 0:
                      res = j.strip('][').split(', ')
                      first_time_2 = float(res[0])
                      fixation_range_container.append(first_time_2 - start_time)
                      start_time = 0
                      a.append(max(fixation_range_container))
                      #e.append(len(a))
                    elif len(fixation_range_container) == 0:
                      a.append(0)
                      #e.append(len(a))
                    #print(index)
                    elif len(fixation_range_container) != 0 and start_time == 0:
                      a.append(max(fixation_range_container))



        #e.append(len(a))
    #demo = {'ColumnA': e, 'ColumnB': d}
    #demo_df = pd.DataFrame(demo)
    data = {'ColumnA': a}
    lg_df = pd.DataFrame(data)
    return lg_df


In [ ]:
def verbalization_lag(df_list1):
    c = 0
    d = list(range(2390))
    e = []
    f = 0
    a = []
    b = 0
    f_end = []
    demo = []
    # fixation_range_container = []

    for df in df_list1:
        b += 1
        last_index_label = df.index.max()
        last_row = df.loc[last_index_label]
        start_time = 0
        end_list = []
        fixation_range_container = []
        for index, row in df.iterrows():
            i = row['Eye movement type']
            j = row['Time Window']

            if i == 1 and not row.equals(last_row):
                res = j.strip('][').split(', ')
                last_time = float(res[1])
                first_time = float(res[0])
                if start_time == 0:
                    start_time = first_time

            if i != 1 and not row.equals(last_row):
                res = j.strip('][').split(', ')
                first_time_2 = float(res[0])
                if start_time != 0:
                    fixation_range_container.append(first_time_2 - start_time)
                    start_time = 0
                    end_list.append(first_time_2)

            if row.equals(last_row):
                if i == 1:
                    res = j.strip('][').split(', ')
                    last_time = float(res[1])
                    first_time = float(res[0])
                    if start_time == 0:
                        fixation_range_container.append(last_time - first_time)
                        end_list.append(last_time)
                        start_time = 0
                        a.append(max(fixation_range_container))
                    else:
                        fixation_range_container.append(last_time - start_time)
                        end_list.append(last_time)
                        a.append(max(fixation_range_container))
                else:
                    if start_time != 0:
                        res = j.strip('][').split(', ')
                        first_time_2 = float(res[0])
                        fixation_range_container.append(first_time_2 - start_time)
                        end_list.append(first_time_2)
                        start_time = 0
                        a.append(max(fixation_range_container))
                    elif len(fixation_range_container) == 0:
                        a.append(0)
                    elif len(fixation_range_container) != 0 and start_time == 0:
                        a.append(max(fixation_range_container))

        ind = fixation_range_container.index(max(fixation_range_container))
        res = last_row['Time Window'].strip('][').split(', ')
        start_p = float(res[0])
        if start_p - end_list[ind] < 0:
            f_end.append(start_p - end_list[ind] + max(fixation_range_container))
        else:
            f_end.append(start_p - end_list[ind] + max(fixation_range_container))

    data = {'ColumnA': f_end}
    lg_df = pd.DataFrame(data)
    return lg_df

In [ ]:
def Temporal_movement(df_list, col_names):
    a = Saccade_fixation_ratio(df_list).reset_index(drop=True)
    b = calculate_avg_a(df_list, col_names).reset_index(drop=True)
    c = longest_fixation(df_list)
    d = verbalization_lag(df_list)
    result = pd.concat([a, b, c, d], axis=1, ignore_index=True)
    result.columns = ['Fixation ratio', 'Saccade ratio', 'Trajectory Length Gaze', 'Longest Fixation range', 'verbalization lag']
    return result


def Gaze_pupil(df_list, col_names_1):
    a = get_std_df(df_list, ['Fixation point X', 'Fixation point Y']).reset_index(drop=True)
    b = avg_col(df_list, 'Fixation point X')
    c = avg_col(df_list, 'Fixation point Y')
    d = get_std_df(df_list, col_names_1).reset_index(drop=True)
    e = absolute_changes(df_list, 'Gaze point 3D X').reset_index(drop=True)
    f = absolute_changes(df_list, 'Gaze point 3D Y').reset_index(drop=True)
    g = absolute_changes(df_list, 'Gaze point 3D Z').reset_index(drop=True)
    h = avg_col(df_list, 'Gaze point 3D X').reset_index(drop=True)
    i = avg_col(df_list, 'Gaze point 3D Y').reset_index(drop=True)
    j = avg_col(df_list, 'Gaze point 3D Z').reset_index(drop=True)
    k = get_std_df(df_list, ['Gaze point X', 'Gaze point Y']).reset_index(drop=True)
    l = avg_col(df_list, 'Gaze point X').reset_index(drop=True)
    m = avg_col(df_list, 'Gaze point Y').reset_index(drop=True)
    n = get_std_df(df_list, ['Gaze direction left X', 'Gaze direction left Y', 'Gaze direction left Z']).reset_index(drop=True)
    o = avg_col(df_list, 'Gaze direction left X').reset_index(drop=True)
    p = avg_col(df_list, 'Gaze direction left Y').reset_index(drop=True)
    q = avg_col(df_list, 'Gaze direction left Z').reset_index(drop=True)
    r = get_std_df(df_list, ['Gaze direction right X', 'Gaze direction right Y', 'Gaze direction right Z']).reset_index(drop=True)
    s = avg_col(df_list, 'Gaze direction right X').reset_index(drop=True)
    t = avg_col(df_list, 'Gaze direction right Y').reset_index(drop=True)
    u = avg_col(df_list, 'Gaze direction right Z').reset_index(drop=True)

    v = get_std_df(df_list, ['Pupil position left X', 'Pupil position left Y', 'Pupil position left Z']).reset_index(drop=True)
    w = avg_col(df_list, 'Pupil position left X').reset_index(drop=True)
    x = avg_col(df_list, 'Pupil position left Y').reset_index(drop=True)
    y = avg_col(df_list, 'Pupil position left Z').reset_index(drop=True)

    z = get_std_df(df_list, ['Pupil position right X', 'Pupil position right Y', 'Pupil position right Z']).reset_index(drop=True)
    z1 = avg_col(df_list, 'Pupil position right X').reset_index(drop=True)
    z2 = avg_col(df_list, 'Pupil position right Y').reset_index(drop=True)
    z3 = avg_col(df_list, 'Pupil position right Z').reset_index(drop=True)
    z4 = get_std_df(df_list, ['Pupil diameter left', 'Pupil diameter right']).reset_index(drop=True)
    z5 = avg_col(df_list, 'Pupil diameter left')
    z6 = avg_col(df_list, 'Pupil diameter right')

    result = pd.concat([a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, z1, z2, z3, z4, z5, z6], axis=1, ignore_index=True)
    new_name = ['SD of Fixation point X', 'SD of Fixation point Y', 'avg of Fixation point X', 'avg of Fixation point Y', \
                'SD of Gaze point 3D X', 'SD of Gaze point 3D Y', 'SD of Gaze point 3D Z', 'Abs change of Gaze point 3D X', 'Abs change of Gaze point 3D Y', 'Abs change of Gaze point 3D Z',\
                'avg of Gaze point 3D X', 'avg of Gaze point 3D Y', 'avg of Gaze point 3D Z',\
                'SD of Gaze point X', 'SD of Gaze point Y', 'avg of Gaze point X', 'avg of Gaze point Y',\
                'SD of Gaze direction left X', 'SD of Gaze direction left Y', 'SD of Gaze direction left Z', 'avg of Gaze direction left X', 'avg of Gaze direction left Y', 'avg of Gaze direction left Z',\
                'SD of Gaze direction right X', 'SD of Gaze direction right Y', 'SD of Gaze direction right Z', 'avg of Gaze direction right X', 'avg of Gaze direction right Y', 'avg of Gaze direction right Z',\
                'SD of Pupil position left X', 'SD of Pupil position left Y', 'SD of Pupil position left Z',\
                'avg of Pupil position left X', 'avg of Pupil position left Y', 'avg of Pupil position left Z',\
                'SD of Pupil position right X', 'SD of Pupil position right Y', 'SD of Pupil position right Z',\
                'avg of Pupil position right X', 'avg of Pupil position right Y', 'avg of Pupil position right Z',\
                'SD of Pupil diameter left', 'SD of Pupil diameter right',\
                'avg of Pupil diameter left', 'avg of Pupil diameter right']
    result.columns = new_name

    return result

def headmovement_orientation(df_list, col_names):
    a = absolute_changes(df_list, 'Gyro X').reset_index(drop=True)
    b = absolute_changes(df_list, 'Gyro Y').reset_index(drop=True)
    c = absolute_changes(df_list, 'Gyro Z').reset_index(drop=True)
    d = avg_col(df_list, 'Gyro X').reset_index(drop=True)
    e = avg_col(df_list, 'Gyro Y').reset_index(drop=True)
    f = avg_col(df_list, 'Gyro Z').reset_index(drop=True)
    g = get_std_df(df_list, col_names).reset_index(drop=True)
    h = calculate_avg_a(df_list, col_names).reset_index(drop=True)
    i = get_std_df(df_list, ['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z']).reset_index(drop=True)
    j = avg_col(df_list, 'Accelerometer X').reset_index(drop=True)
    k = avg_col(df_list, 'Accelerometer Y').reset_index(drop=True)
    l = avg_col(df_list, 'Accelerometer Z').reset_index(drop=True)
    m = calculate_avg_a(df_list, ['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z']).reset_index(drop=True)
    result = pd.concat([a, b, c, d, e, f, g, h, i, j, k, l, m], axis=1, ignore_index=True)
    new_name = ['abs_gyro_x', 'abs_gyro_y', 'abs_gyro_z',\
                'avg of Gyro X', 'avg of Gyro Y', 'avg of Gyro Z',\
                 'SD of Gyro X', 'SD of Gyro Y', 'SD of Gyro Z', 'Trajectory Length Gyro',\
                'SD of Accelerometer X', 'SD of Accelerometer Y', 'SD of Accelerometer Z',\
                'avg of Accelerometer X', 'avg of Accelerometer Y', 'avg of Accelerometer Z', 'Trajectory Length Accelerometer']
    result.columns = new_name

    return result

In [ ]:
def remove_useless_window(df_list1, df_list2):
  indexes_to_remove = []
  indexes_to_remove_1 = []
  a = longest_fixation(df_list1)
  b = longest_fixation(df_list2)
  for i in range(len(a['ColumnA'])):
    if a['ColumnA'][i] == 0:
      indexes_to_remove.append(i)
  for i in range(len(b['ColumnA'])):
    if b['ColumnA'][i] == 0:
      indexes_to_remove_1.append(i)
  new_window_2s = [df for i, df in enumerate(df_list1) if i not in indexes_to_remove]
  new_neg_window_2s = [df for i, df in enumerate(df_list2) if i not in indexes_to_remove_1]
  return new_window_2s, new_neg_window_2s


In [ ]:
window_1s, neg_window_1s = parse_data(X_filled, 1, include=False)
window_2s, neg_window_2s = parse_data(X_filled, 2, include=False)
window_3s, neg_window_3s = parse_data(X_filled, 3, include=False)
window_4s, neg_window_4s = parse_data(X_filled, 4, include=False)
window_5s, neg_window_5s = parse_data(X_filled, 5, include=False)
window_0_5s, neg_window_0_5s = parse_data(X_filled, 0.5, include=False)

In [ ]:
del neg_window_0_5s[2810:2833]

In [ ]:
new_window_1s, new_neg_window_1s = remove_useless_window(window_1s, neg_window_1s)
new_window_2s, new_neg_window_2s = remove_useless_window(window_2s, neg_window_2s)
new_window_3s, new_neg_window_3s = remove_useless_window(window_3s, neg_window_3s)
new_window_4s, new_neg_window_4s = remove_useless_window(window_4s, neg_window_4s)
new_window_5s, new_neg_window_5s = remove_useless_window(window_5s, neg_window_5s)
new_window_0_5s, new_neg_window_0_5s = remove_useless_window(window_0_5s, neg_window_0_5s)

####define response variable

interest level == nan 0

interest level == 0 1

interest level >= 1 2

In [ ]:
def response_cta(df_list1, df_list2):
  a = []
  for df in df_list1:
    for i in df['interest_level']:
      if i == 0:
        a.append(1)
      elif i >= 1:
        a.append(2)
  df2 = pd.DataFrame(a, columns=['Response'])
  length2 = len(df_list2)
  df3 = pd.DataFrame(0, index=range(length2), columns=['Response'])
  df_concat = pd.concat([df2, df3])
  df_concat = df_concat.reset_index(drop=True)
  return df_concat


distraction 1

attention 2

In [ ]:
def response_dis(df_list1, df_list2):
  a = []
  for df in df_list1:
    for i in df['is_distraction']:
      if i == 'False':
        a.append(2)
      elif i == 'True':
        a.append(1)
  df2 = pd.DataFrame(a, columns=['Response'])
  length2 = len(df_list2)
  df3 = pd.DataFrame(0, index=range(length2), columns=['Response'])
  df_concat = pd.concat([df2, df3])
  df_concat = df_concat.reset_index(drop=True)
  return df_concat

###conduct X & y

In [ ]:
def get_x_y_c(df1, df2):
    a1 = Temporal_movement(df1, ['Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z'])
    a2 = Temporal_movement(df2, ['Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z'])
    b1 = Gaze_pupil(df1, ['Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z'])
    b2 = Gaze_pupil(df2, ['Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z'])
    c1 = headmovement_orientation(df1, ['Gyro X', 'Gyro Y', 'Gyro Z'])
    c2 = headmovement_orientation(df2, ['Gyro X', 'Gyro Y', 'Gyro Z'])
    X1 = con_df(a1, b1, c1)
    X2 = con_df(a2, b2, c2)
    X = pd.concat([X1, X2])
    X = X.reset_index(drop=True)
    y = response_cta(df1, df2)
    return X, y

In [ ]:
def get_x_y_d(df1, df2):
    a1 = Temporal_movement(df1, ['Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z'])
    a2 = Temporal_movement(df2, ['Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z'])
    b1 = Gaze_pupil(df1, ['Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z'])
    b2 = Gaze_pupil(df2, ['Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z'])
    c1 = headmovement_orientation(df1, ['Gyro X', 'Gyro Y', 'Gyro Z'])
    c2 = headmovement_orientation(df2, ['Gyro X', 'Gyro Y', 'Gyro Z'])
    X1 = con_df(a1, b1, c1)
    X2 = con_df(a2, b2, c2)
    X = pd.concat([X1, X2])
    X = X.reset_index(drop=True)
    y = response_dis(df1, df2)
    return X, y

In [ ]:
#X_0_2s, y_0_2s = get_x_y(window_0_2s, neg_window_0_2s)
X_0_5s_c, y_0_5s_c = get_x_y_c(new_window_0_5s, new_neg_window_0_5s)
X_1s_c, y_1s_c = get_x_y_c(new_window_1s, new_neg_window_1s)
#X_1_5s, y_1_5s = get_x_y(window_1_5s, neg_window_1_5s)
X_2s_c, y_2s_c = get_x_y_c(new_window_2s, new_neg_window_2s)
#X_2_5s, y_2_5s = get_x_y(window_2_5s, neg_window_2_5s)
X_3s_c, y_3s_c = get_x_y_c(new_window_3s, new_neg_window_3s)
#X_3_5s, y_3_5s = get_x_y(window_3_5s, neg_window_3_5s)
X_4s_c, y_4s_c = get_x_y_c(new_window_4s, new_neg_window_4s)
X_5s_c, y_5s_c = get_x_y_c(new_window_5s, new_neg_window_5s)

In [ ]:
#X_0_2s, y_0_2s = get_x_y(window_0_2s, neg_window_0_2s)
X_0_5s_d, y_0_5s_d = get_x_y_d(new_window_0_5s, new_neg_window_0_5s)
X_1s_d, y_1s_d = get_x_y_d(new_window_1s, new_neg_window_1s)
#X_1_5s, y_1_5s = get_x_y(window_1_5s, neg_window_1_5s)
X_2s_d, y_2s_d = get_x_y_d(new_window_2s, new_neg_window_2s)
#X_2_5s, y_2_5s = get_x_y(window_2_5s, neg_window_2_5s)
X_3s_d, y_3s_d = get_x_y_d(new_window_3s, new_neg_window_3s)
#X_3_5s, y_3_5s = get_x_y(window_3_5s, neg_window_3_5s)
X_4s_d, y_4s_d = get_x_y_d(new_window_4s, new_neg_window_4s)
X_5s_d, y_5s_d = get_x_y_d(new_window_5s, new_neg_window_5s)